# FinalTask. PredictWinner

Dota 2 — многопользовательская компьютерная игра жанра MOBA. Игроки играют между собой матчи. В каждом матче, как правило, участвует 10 человек. Матчи формируются из живой очереди, с учётом уровня игры всех игроков. Перед началом игры игроки автоматически разделяются на две команды по пять человек. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника, трон.

Вам нужно построить модель, которая по данным о первых пяти минутах матча будет предсказывать его исход — то есть определять команду-победителя.

Чтобы выполнить это задание, вам необходимо провести ряд исследований, сравнить несколько алгоритмов машинного обучения и проверить эффект от ряда манипуляций с признаками.

Вам необходимо провести описанные в документе final-statement.html (или final-statement.ipynb) два этапа исследования (для двух подходов к решению задачи), написать по результатам каждого этапа небольшой отчет (ниже указаны вопросы, ответы на которые должны содержаться в отчете), и предоставить для ревью данный отчет и код, с помощью которого вы выполнили задание.

Не забывайте, что в выборке есть признаки, которые "заглядывают в будущее" — они помечены в описании данных как отсутствующие в тестовой выборке. Их прямое использование в модели приведет к переобучению, поэтому не забудьте исключить их из выборки.

## Подход 1: градиентный бустинг "в лоб"

Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [1]:
import pandas

features = pandas.read_csv('./features.csv', index_col='match_id')
X_test = pandas.read_csv('./features_test.csv', index_col='match_id')

# drop all features that can "look into the future" from train set
X_train = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
                         'barracks_status_radiant', 'barracks_status_dire'], axis=1)

In [2]:
X_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [4]:
X_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [3]:
# checking for missing values in train set
for column in X_train:
    print("Column {}: number of missing values = {}".format(column, X_train.values.__len__() - X_train[column].count()))

Column start_time: number of missing values = 0
Column lobby_type: number of missing values = 0
Column r1_hero: number of missing values = 0
Column r1_level: number of missing values = 0
Column r1_xp: number of missing values = 0
Column r1_gold: number of missing values = 0
Column r1_lh: number of missing values = 0
Column r1_kills: number of missing values = 0
Column r1_deaths: number of missing values = 0
Column r1_items: number of missing values = 0
Column r2_hero: number of missing values = 0
Column r2_level: number of missing values = 0
Column r2_xp: number of missing values = 0
Column r2_gold: number of missing values = 0
Column r2_lh: number of missing values = 0
Column r2_kills: number of missing values = 0
Column r2_deaths: number of missing values = 0
Column r2_items: number of missing values = 0
Column r3_hero: number of missing values = 0
Column r3_level: number of missing values = 0
Column r3_xp: number of missing values = 0
Column r3_gold: number of missing values = 0
Col

Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
# fill NA in train and test sets with 0
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [6]:
X_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [7]:
X_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


Какой столбец содержит целевую переменную? Запишите его название.

In [8]:
# target feature
y = features['radiant_win']

In [10]:
y.head()

match_id
0    1
1    1
2    0
3    0
4    0
Name: radiant_win, dtype: int64

Целевая переменная: radiant_win

Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import datetime

def gradient_boosting(n_estimators, X, y, max_depth=3):
    start_time = datetime.datetime.now()

    gbc = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=241)
    kf = KFold(n_splits=5, shuffle=True)

    roc_score = np.empty([kf.n_splits, 1])

    # cross-validation
    for k, (train, test) in enumerate(kf.split(X, y)):
        x_train, y_train = X.iloc[train], y.iloc[train]
        x, y_test = X.iloc[test], y.iloc[test]

        gbc.fit(x_train, y_train)
        pred = gbc.predict_proba(x)[:, 1]

        # saving roc_score for each folds
        roc_score[k] = roc_auc_score(y_test, pred)

    print("n_estimators = {}\nAUC-ROC = {}".format(gbc.n_estimators, np.mean(roc_score))) # roc_score as mean of each fold
    print('Time elapsed:', (datetime.datetime.now() - start_time).seconds)

    # fit classifier on full train set
    gbc.fit(X, y)

    return gbc

In [12]:
gradient_boosting(10, X_train, y)
gradient_boosting(20, X_train, y)
gradient_boosting(30, X_train, y)
gradient_boosting(40, X_train, y)
gradient_boosting(50, X_train, y)
gradient_boosting(60, X_train, y)
gradient_boosting(70, X_train, y)
gradient_boosting(80, X_train, y)

n_estimators = 10
AUC-ROC = 0.665904287796
('Time elapsed:', 27)
n_estimators = 20
AUC-ROC = 0.682395702551
('Time elapsed:', 54)
n_estimators = 30
AUC-ROC = 0.689152076826
('Time elapsed:', 85)
n_estimators = 40
AUC-ROC = 0.694065760227
('Time elapsed:', 101)
n_estimators = 50
AUC-ROC = 0.697514977863
('Time elapsed:', 140)
n_estimators = 60
AUC-ROC = 0.700544497544
('Time elapsed:', 157)
n_estimators = 70
AUC-ROC = 0.702352855824
('Time elapsed:', 197)
n_estimators = 80
AUC-ROC = 0.703562886235
('Time elapsed:', 216)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=80,
              presort='auto', random_state=241, subsample=1.0, verbose=0,
              warm_start=False)

В отчете по данному этапу вы должны ответить на следующие вопросы:
* Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

Ответ:  
first_blood_time  
first_blood_team  
first_blood_player1  
first_blood_player2  
(все 4 признака относятся к событию "first_blood", пропущенные значения означают, что событие "первая кровь" не успело произойти за первые 5 минут)  

radiant_bottle_time -- предмет "bottle" не был приобретен командой Radiant в первые 5 минут матча  
radiant_courier_time -- предмет "courier" не был приобретен командой Radiant в первые 5 минут матча  
radiant_flying_courier_time -- предмет "flying_courier" не был приобретен командой Radiant в первые 5 минут матча  
radiant_first_ward_time -- "наблюдатель" не был установлен в первые 5 минут матча командой Radiant  
dire_bottle_time -- предмет "bottle" не был приобретен командой Dire в первые 5 минут матча  
dire_courier_time -- предмет "courier" не был приобретен командой Dire в первые 5 минут матча  
dire_flying_courier_time -- предмет "flying_courier" не был приобретен командой Dire в первые 5 минут матча  
dire_first_ward_time -- "наблюдатель" не был установлен в первые 5 минут матча командой Dire  

* Как называется столбец, содержащий целевую переменную?

Ответ: radiant_win  

* Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.  

Ответ: 93 секунды. AUC-ROC = 0.69  

* Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?  

Ответ: При 80 деревьях AUC-ROC = 0.70 (за 251 секунду), маловероятно, что с увеличением количества деревьев можно добиться значительно лучшего качества. Для ускорения обучения можно сократить глубину деревьев, кроме того есть возможность изменять параметр learning_rate, который отвечает за скорость обучения.

## Подход 2: логистическая регрессия¶

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.
Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


def logistic_regression(X, y, C):
    start_time = datetime.datetime.now()

    scaler = StandardScaler()

    clf = LogisticRegression(C=C, random_state=241)
    kf = KFold(n_splits=5, shuffle=True)

    roc_score = np.empty([kf.n_splits, 1])

    # cross-validation
    for k, (train, test) in enumerate(kf.split(X, y)):

        # scaling each set
        x_train, y_train = scaler.fit_transform(X.iloc[train]), y.iloc[train]
        x_test, y_test = scaler.transform(X.iloc[test]), y.iloc[test]

        clf.fit(x_train, y_train)
        pred = clf.predict_proba(x_test)[:, 1]

        # saving roc_score for each folds
        roc_score[k] = roc_auc_score(y_test, pred)

    print("AUC-ROC = {}".format(np.mean(roc_score))) # roc_score as mean of each fold
    print('Time elapsed:', (datetime.datetime.now() - start_time).seconds)

    # scaling full train set
    X = scaler.fit_transform(X)

    # fit classifier on full train set
    clf.fit(X, y)

    return clf, scaler

In [35]:
from sklearn.model_selection import GridSearchCV


def search_c(X, y):
    param_grid = {'C': [1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3]}
    scoring = {'AUC': "roc_auc"}

    kf = KFold(n_splits=5, shuffle=True)

    # normalization of features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # searching best regularization parameter by GridSearch
    clf = GridSearchCV(LogisticRegression(random_state=241), param_grid=param_grid, scoring=scoring, refit='AUC', cv=kf)
    clf.fit(X, y)

    print("Best params: {}".format(clf.best_params_))

    return clf.best_params_.values()[0]

In [15]:
# searching for best regularization parameter
best_C = search_c(X_train, y)
logistic_regression(X_train, y, best_C)

Best params: {'C': 0.01}
AUC-ROC = 0.716188202091
('Time elapsed:', 14)


(LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 StandardScaler(copy=True, with_mean=True, with_std=True))

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [16]:
# exclude categorical features
cat_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                            'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
new_X_train = X_train.drop(cat_features, axis=1)
new_X_test = X_test.drop(cat_features, axis=1)

In [17]:
# searching for best regularization parameter
best_C = search_c(new_X_train, y)
logistic_regression(new_X_train, y, best_C)

Best params: {'C': 0.01}
AUC-ROC = 0.7167152628
('Time elapsed:', 11)


(LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 StandardScaler(copy=True, with_mean=True, with_std=True))

На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [18]:
# renumbering cell values in range(N+1)
cells = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
         'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

X_train[cells] = X_train[cells].rank(method='dense')
X_test[cells] = X_test[cells].rank(method='dense')

In [19]:
X_train[cells]

,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
match_id,,,,,,,,,,
0,11.0,66.0,28.0,20.0,104.0,4.0,41.0,21.0,36.0,83.0
1,41.0,48.0,66.0,36.0,25.0,38.0,87.0,78.0,7.0,12.0
2,32.0,97.0,20.0,26.0,4.0,22.0,65.0,85.0,28.0,79.0
3,28.0,29.0,74.0,36.0,40.0,95.0,47.0,15.0,101.0,20.0
4,13.0,26.0,29.0,71.0,92.0,25.0,68.0,22.0,24.0,8.0
5,11.0,20.0,27.0,24.0,64.0,54.0,51.0,3.0,72.0,47.0
8,8.0,56.0,7.0,21.0,35.0,22.0,29.0,71.0,100.0,46.0
9,34.0,15.0,82.0,28.0,100.0,99.0,24.0,25.0,43.0,95.0
11,17.0,90.0,52.0,71.0,29.0,89.0,95.0,34.0,19.0,7.0


In [20]:
X_test[cells]

,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
match_id,,,,,,,,,,
6,92.0,101.0,51.0,52.0,56.0,25.0,88.0,20.0,28.0,38.0
7,20.0,6.0,64.0,91.0,42.0,53.0,33.0,92.0,98.0,5.0
10,108.0,25.0,28.0,11.0,40.0,90.0,22.0,10.0,48.0,53.0
13,26.0,90.0,73.0,51.0,2.0,107.0,24.0,76.0,61.0,48.0
16,38.0,92.0,29.0,84.0,101.0,14.0,63.0,69.0,3.0,95.0
18,96.0,71.0,30.0,9.0,62.0,58.0,42.0,98.0,5.0,7.0
19,96.0,92.0,49.0,50.0,108.0,20.0,84.0,66.0,34.0,100.0
24,105.0,96.0,25.0,98.0,90.0,45.0,51.0,12.0,7.0,29.0
33,56.0,38.0,32.0,47.0,2.0,53.0,86.0,33.0,29.0,28.0


In [21]:
def number_of_unique_heroes(X):
    N = pandas.unique(X[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                         'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].values.ravel('K')).size
    print("Number of unique hero ID: {}".format(N))

    return N

In [22]:
# checking for number of unique heroes in train and test sets
N_train = number_of_unique_heroes(X_train)
N_test = number_of_unique_heroes(X_test)

Number of unique hero ID: 108
Number of unique hero ID: 108


Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [23]:
def convert_categorical_features(N, X):
    # N -- number of different heros
    X_pick = np.zeros((X.shape[0], N))

    for i, match_id in enumerate(X.index):
        for p in xrange(5):
            X_pick[i, int(X.loc[match_id, 'r%d_hero' % (p + 1)] - 1)] = 1
            X_pick[i, int(X.loc[match_id, 'd%d_hero' % (p + 1)] - 1)] = -1

    # creating keys for dict
    keys = []
    for i in xrange(1, N + 1):
        keys.append('hero_%d' % i)

    # creating dict with None values
    heroes = dict.fromkeys(keys)

    # adding empty arrays as values into dict
    for hero in heroes.keys():
        heroes[hero] = []

    # filling arrays by X_pick values
    for j in range(X_pick.__len__()):
        for i in range(1, N + 1):
            heroes['hero_%d' % i].append(X_pick[j][i - 1])

    # creating dataframe from dict
    features_heroes = pandas.DataFrame(heroes, index=X.index)

    return features_heroes

In [24]:
# convertation of categorical features in train and test sets
heroes_train = convert_categorical_features(N_train, X_train)
heroes_test = convert_categorical_features(N_test, X_test)

In [27]:
heroes_train.head()

,hero_1,hero_10,hero_100,hero_101,hero_102,hero_103,hero_104,hero_105,hero_106,hero_107,...,hero_90,hero_91,hero_92,hero_93,hero_94,hero_95,hero_96,hero_97,hero_98,hero_99
match_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
heroes_test.head()

,hero_1,hero_10,hero_100,hero_101,hero_102,hero_103,hero_104,hero_105,hero_106,hero_107,...,hero_90,hero_91,hero_92,hero_93,hero_94,hero_95,hero_96,hero_97,hero_98,hero_99
match_id,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
10,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0


In [29]:
# concatenation converted features and rest set
new_X_train = pandas.concat([new_X_train, heroes_train], axis=1, join_axes=[new_X_train.index])
new_X_test = pandas.concat([new_X_test, heroes_test], axis=1, join_axes=[new_X_test.index])

In [30]:
new_X_train.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,hero_90,hero_91,hero_92,hero_93,hero_94,hero_95,hero_96,hero_97,hero_98,hero_99
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
new_X_test.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,hero_90,hero_91,hero_92,hero_93,hero_94,hero_95,hero_96,hero_97,hero_98,hero_99
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,4,1103,1089,8,0,1,9,3,1183,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1430293357,2,556,570,1,0,0,9,4,1194,...,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
10,1430301774,2,751,808,1,0,0,13,2,421,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,1430323933,3,708,903,1,1,1,11,2,672,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1430331112,4,1259,661,4,0,0,9,5,1703,...,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0


Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [36]:
# searching for best regularization parameter
best_C = search_c(new_X_train, y)
clf, scaler = logistic_regression(new_X_train, y, best_C)

Best params: {'C': 0.01}
AUC-ROC = 0.751922682211
('Time elapsed:', 26)


Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [37]:
# scaling test set
scaled_X = scaler.transform(new_X_test)

pred = clf.predict_proba(scaled_X)[:, 1]

print("min = {}".format(pred.min()))
print("max = {}".format(pred.max()))

min = 0.00849095194724
max = 0.996277624036


В отчете по данному этапу вы должны ответить на следующие вопросы:

* Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Ответ: AUC-ROC = 0.72. Это чуть лучше, чем качество градиентного бустинга. Возможно, на таких данных логистическая регрессия работает лучше, и может давать лучшее качество модели. Кроме того, логистическая регрессия работает быстрее (15 секунд).  

* Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Ответ: AUC-ROC = 0.72. Отличие только в десятитысячной доле (чуть лучше). Похоже, что удаление категориальных признаков не влияет на улучшение качества, но нельзя сказать также, что качество стало хуже.  

* Сколько различных идентификаторов героев существует в данной игре?

Ответ: 108  

* Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Ответ: AUC-ROC = 0.75. Заметно улучшение качества. Чтобы использовать категориальные признаки в обучении, нужно преобразовать их в числовые, т.к в своем первоначальном виде на них нельзя ввести порядок. В данном случае категориальные признаки являются важными признаками, т.к герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие.  

* Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

Ответ: min = 0.008, max = 0.996